In [1]:
import pandas as pd
import panel as pn
import numpy as np
import hvplot.pandas
import folium
from panel.interact import interact
pn.extension('tabulator')
pn.extension

panel.config.panel_extension

In [2]:
df = pd.read_csv('merged_data_5.csv', sep=',')


In [3]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('merged_data_5.csv', sep=',')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [4]:
#df.shape

In [5]:
#df.isnull().sum()

In [6]:
cero_happiness = df['Happiness'].isnull() == True
df.loc[cero_happiness,:]

,Unnamed: 0,Year,Country,Region,Happiness_Rank,Happiness,Explained_by_Trust,Explained_by_GDP,Explained_by_Freedom,Explained_by_Generosity,Explained_by_Health,Explained_by_Social_support,Dystopia_Residual,ISO_Code
283,283,2022,xx,NaN,147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#df.info()

In [8]:
#df.nunique()

In [9]:
#df.Region.value_counts()

In [10]:
#glob_rank = df.groupby('Country')[['Happiness_Rank','Year']].min()
#glob_rank

In [11]:
#glob_hap_max = df.groupby('Year')['Happiness'].max()
#glob_hap_mean = df.groupby('Year')['Happiness'].mean()
#glob_hap_min = df.groupby('Year')['Happiness'].min() 

In [12]:
#country_happiness = df.groupby('Country')[['Year','Happiness','Happiness_Rank']]

In [13]:
#rank_1 = df['Happiness_Rank'] == 1
#df.loc[rank_1,:]

In [14]:
#rank_2 = df['Happiness_Rank'] == 2
#df.loc[rank_2,:]

In [15]:
df['Happiness_Rank'].max()

158

In [16]:
df['Happiness'].min()

1.859

# Make df interactive

In [17]:
idf = df.interactive()

In [18]:
idf.head(1)

# Define Panel Widgets 

In [19]:
from bokeh.models.formatters import PrintfTickFormatter

rank_slider = pn.widgets.IntSlider(name='Happiness Rank', start=1, end=158, step=1, value=1)
rank_slider

IntSlider(end=158, name='Happiness Rank', start=1, value=1)

In [20]:
# Radio buttons for CO2 measures
yaxis_Happiness = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Happiness'],
    button_type='success'
)

In [21]:
year = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022','2023']

happiness_pipeline = (
    idf[
        (idf.Happiness_Rank <= rank_slider) #&
        #(idf.Year.isin(year))
    ]
    .groupby(['Country', 'Year']).min()
    #.to_frame()
    .reset_index()
    .sort_values(by='Happiness_Rank', ascending=True)  
    #.reset_index(drop=True)
)

In [22]:
happiness_pipeline[-10:]

In [23]:
happiness_plot = happiness_pipeline.hvplot(x='Year', by='Country', y=yaxis_Happiness, line_width=1.5, title="test", grid=True, hover_cols='Happiness_Rank')
happiness_plot

C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


In [24]:
rank_slider.value

1

In [25]:
rank = idf['Happiness_Rank'] == rank_slider.value
#no_1_2015_2023 = 
idf.loc[rank,:].sort_values(by='Year', ascending=True)


In [26]:
year = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022','2023']

happiness_pipeline_test = (
    idf[
        (idf.Happiness_Rank <= rank_slider) #&
        #(idf.Year.isin(year))
    ]
    .sort_values(by='Year', ascending=True)
    #.groupby(['Country', 'Year']).min()
    #.to_frame()
    #.reset_index()
    #.sort_values(by='Happiness_Rank', ascending=True)  
    #.reset_index(drop=True)
)

#rank = idf['Happiness_Rank'] == rank_slider.value
#no_1_2015_2023 = 
#idf.loc[rank,:].sort_values(by='Year', ascending=True)
happiness_pipeline_test

In [27]:
yaxis_Happiness_test = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Happiness'],
    button_type='success'
)

In [28]:
happiness_plot_test = happiness_pipeline_test.hvplot.line(x='Year', 
                                                     by='Country', 
                                                     y=yaxis_Happiness_test, 
                                                     line_width=1.5, title="test", 
                                                     grid=True, 
                                                     hover_cols='Happiness_Rank',
                                                     cmap='Dark2',
                                                         width=800,
                                                         height=400)
happiness_plot_test

C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\User\anaconda3\Lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
